# Chapters 4 and 5 -- ArgMax, SoftMax, and Cross Entropy

In this tutorial, we will use **[PyTorch](https://pytorch.org/) and [Lightning](https://www.lightning.ai/)** to create and optimize a simple neural network with multiple inputs and outputs that uses the `SoftMax()` function during training, the the one in the picture shown below.

<img src="./images/chapter_4_softmax.png" alt="a neural network with multiple inputs and outputs and softmax" style="width: 800px;">

and the `ArgMax()` function during inference.

<img src="./images/chapter_4_argmax.png" alt="a neural network with multiple inputs and outputs and argmax" style="width: 800px;">


In this tutorial, we will:

- Import and Format Data and then Build a DataLoader From Scratch
- Build a Neural Network with Multiple Inputs and Outputs and the SoftMax() function
- Train a Neural Network with Multiple Inputs and Outputs and the SoftMax() function
- Make Predictions with New Data Using the Trained Model and the ArgMax() function

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import lightning as L
from sklearn.model_selection import train_test_split

## Build a DataLoader

As in the previous chapter, we will use the **[Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)**, which we will import from a comma-separated (CSV) text file. We will then proceed exactly the same, and prepare the training data into a **DataLoader**, which we can use to train the neural network.

In [2]:
# --- Import the scaler and label encoder from sklearn ---
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# --- Load and Define X and y ---
path = "./iris.txt"
# Assign column names since the file has no header
column_names = [
    'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

df = pd.read_csv(
    path, sep=",", names=column_names)

# --- Define X and y ---
input_values = df[['petal_width', 'sepal_width']]
label_values = df['class']

# --- SPLIT THE DATA FIRST ---
# (This is best practice)
input_train, input_test, label_train_str, label_test_str = train_test_split(
    input_values,
    label_values,           # split the original label strings
    test_size=0.25,
    stratify=label_values,  # stratify on the label strings
    random_state=42
)

# --- SKLEARN PREPROCESSING ---
# Initialize and fit the LabelEncoder
le = LabelEncoder()
le.fit(label_train_str)  # Fit *only* on the training labels

# Transform *both* sets
label_train_num = le.transform(label_train_str)
label_test_num = le.transform(label_test_str)

# (We can also use le.fit_transform(label_train_str) for the first one)

# Initialize and fit the MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(input_train)
input_train_scaled = scaler.transform(input_train)
input_test_scaled = scaler.transform(input_test)

# --- PYTORCH CONVERSION ---
# Convert scaled data to tensors
input_train_tensors = torch.tensor(input_train_scaled, dtype=torch.float32)
input_test_tensors = torch.tensor(input_test_scaled, dtype=torch.float32)

# Convert encoded labels to tensors
label_train_tensors = torch.tensor(label_train_num)
label_test_tensors = torch.tensor(label_test_num)

# One-hot encode the numeric labels
# The model's output layer will produce float32 predictions. 
# To calculate the loss, PyTorch requires the predictions 
# and the labels to be the same data type.
one_hot_label_train = F.one_hot(label_train_tensors).type(torch.float32)
one_hot_label_test = F.one_hot(label_test_tensors).type(torch.float32)

# Create TensorDataset and DataLoader as before
train_dataset = TensorDataset(input_train_tensors, one_hot_label_train)
train_dataloader = DataLoader(train_dataset)

## Building a neural network

Building a neural network with PyTorch means creating a new class. And to make it easy to train the neural network, this class will inherit from `LightningModule`.

Our new class will have the following methods:

- `__init__()` to initialize the Weights and Biases and keep track of a few other housekeeping things.
- `forward()` to make a forward pass through the neural network.
- `configure_optimizers()` to configure the optimizer. There are lots of optimizers to choose from, but in this tutorial, we'll change things up and use `Adam`.
- `training_step()` to pass the training data to `forward()`, calculate the loss and keep track of the loss values in a log file.

Also, for reference, here is a picture of the neural network we want to create is more completed than in the previous chapter, in particular with the addition of a `SoftMax` function in the output layer, and therefore the need for another cost function:

<img src="./images/chapter_4_softmax.png" alt="a neural network with multiple inputs and outputs and softmax" style="width: 800px;">

As we can see in the picture, our neural network has 2 inputs, one for Petal Width and one for Sepal Width, a single hidden layer with two **[ReLU](https://youtu.be/68BZ5f7P94E)** activation functions, and 3 outputs, one for each species of iris.

So, given this specification for this neural network, let's code it in a new class called `MultipleInsOuts`.

In [3]:
class MultipleInsOuts(L.LightningModule):

    def __init__(self):
        super().__init__()

        L.seed_everything(seed=42)

        # We can set the net that will be used by the forward() method here,
        # also the default for the bias argument is True
        self.net = nn.Sequential(
            nn.Linear(in_features=2, out_features=2),  # Linear
            nn.ReLU(),                                 # ReLU
            nn.Linear(in_features=2, out_features=3),  # Linear
        )

        # We'll use Cross Entropy to calculate the loss between what the
        # neural network's predictions and actual, or known, species for
        # each row in the dataset.
        # NOTE: nn.CrossEntropyLoss applies a SoftMax function to the values
        # we give it, so we don't have to do that oursevles. However,
        # when we use this neural network (after it has been trained), we'll
        # have to remember to apply a SoftMax function to the output.
        
        self.loss = nn.CrossEntropyLoss()


    def forward(self, input):
        # We change the method slightly compared to the previous chapter

        return self.net(input)


    def configure_optimizers(self):

        return Adam(self.parameters(), lr=0.001)


    def training_step(self, batch):
        inputs, labels = batch
        outputs = self.forward(inputs)
        loss = self.loss(outputs, labels)

        return loss

## Training the Neural Network

Training our new neural network means we create a model from the new class, `MultipleInsOuts` and then create a **Lightning Trainer**, `L.Trainer`, and use it to optimize the parameters.

In [4]:
model = MultipleInsOuts()

# Let's print the parameters of the neural network
model.state_dict()

Seed set to 42


OrderedDict([('net.0.weight',
              tensor([[ 0.5406,  0.5869],
                      [-0.1657,  0.6496]])),
             ('net.0.bias', tensor([-0.1549,  0.1427])),
             ('net.2.weight',
              tensor([[-0.3443,  0.4153],
                      [ 0.6233, -0.5188],
                      [ 0.6146,  0.1323]])),
             ('net.2.bias', tensor([0.5224, 0.0958, 0.3410]))])

We already see that the neural network initiated here is the same as in the previous chapter, because we use the same configuration with the same seed number. What changes really is the output throught the SoftMax function, and therefore the training process that requires another cost function for minimization.

We will start with 10 epochs, complete runs through our training data. This may be enough to successfully optimize all of the parameters, but it might not. We'll find out later in the tutorial when we make a graph of how the loss values change during training.

In [5]:
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, train_dataloaders=train_dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Sébastien\Documents\data_science\machine_learning\statsquest_neural_networks\.env\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name | Type             | Params | Mode 
---------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


The table differs slightly from the previous chapter, though the information are similar. Now the layers are not separated but grouped into a 'net'. Also, the loss function reflects that we are using a SoftMax function, which necessitates 'CrossEntropyLoss'.

Now, let's see if the predictions are any good. We can do this by seeing how well it predicts the testing data. We'll start by running `input_test_tensors` through the neural network and saving the output `predictions`.

In [6]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)
predictions[:4]

tensor([[ 0.9644, -0.1377,  0.0748],
        [ 0.0507,  0.5664,  0.8702],
        [ 0.1509,  0.4944,  0.6980],
        [ 0.1030,  0.5353,  0.6764]], grad_fn=<SliceBackward0>)

We can determine which species was predicted in `predictions` by selecting the index in each row that corresponding to the largest value, and we do that with `torch.argmax()`. `torch.argmax()` returns a tensor that contains the indices with the largest values for each row.

In [7]:
# Select the output with highest value...
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:4] # print out the first 4 predictions

tensor([0, 2, 2, 2])

In the first row index 0 had the largest value. Thus, the first prediction corresponds to **Setosa**. The second, third, and fourth rows predicted 2, which corresponds to **Virginica**.

Now, let's compare what the neural network predicted in `predicted_labels` to the known values in `label_test` and calculate the percentage of correct predictions. We do this by adding up the number of times an element in `predicted_labels` equals the corresponding element in `label_test` and dividing by the number of elements in `predicted_labels`.

In [8]:
torch.sum(
    torch.eq(
        label_test_tensors,
        predicted_labels
    )) / len(predicted_labels)

tensor(0.6579)

And we see that our neural network only correctly predicts 65.8% of the testing data. This isn't very good. So, let's add an additional **90** epochs to the training.

In [9]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path

# First, create a new Lightning Trainer
trainer = L.Trainer(max_epochs=100)
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    ckpt_path=path_to_checkpoint)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at c:\Users\Sébastien\Documents\data_science\machine_learning\statsquest_neural_networks\chapter_04\lightning_logs\version_9\checkpoints\epoch=9-step=1120.ckpt
c:\Users\Sébastien\Documents\data_science\machine_learning\statsquest_neural_networks\.env\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:445: The dirpath has changed from 'c:\\Users\\Sébastien\\Documents\\data_science\\machine_learning\\statsquest_neural_networks\\chapter_04\\lightning_logs\\version_9\\checkpoints' to 'c:\\Users\\Sébastien\\Documents\\data_science\\machine_learning\\statsquest_neural_networks\\chapter_04\\lightning_logs\\version_10\\checkpoi

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


Now, let's run the testing data through the network and calculate the accuracy. We'll do this just like we did before.

In [10]:
predicted_labels = torch.argmax(
    model(input_test_tensors),
    dim=1)

torch.sum(
    torch.eq(
        label_test_tensors,
        predicted_labels
    )) / len(predicted_labels)

tensor(0.9474)

After 100 training epochs, we correctly classified almost 95% of the testing data using the NN configured as follows.

In [11]:
model.state_dict()

OrderedDict([('net.0.weight',
              tensor([[ 3.5468,  0.2525],
                      [-1.9557,  1.4759]])),
             ('net.0.bias', tensor([-0.3422,  1.2418])),
             ('net.2.weight',
              tensor([[-4.2038,  3.1493],
                      [ 0.1572,  0.3311],
                      [ 1.8735, -3.2946]])),
             ('net.2.bias', tensor([ 0.4556,  0.9597, -0.6117]))])

Remind that in PyTorch's `nn.Linear` the weight matrix is always structured as `[number_of_outputs, number_of_inputs]`. Therefore:

- each ROW in the tensor corresponds to one output node
- each COLUMN in the tensor corresponds to one input node

For example:

```text
Inputs:
           col 0 (from I0)   col 1 (from I1)
        [[   3.5468       ,     0.2525      ],  <-- row 0 (to H0)
         [  -1.9557       ,     1.4759      ]]  <-- row 1 (to H1)
Outputs:
```

For the weights between the hidden and the output layers:

```text
col 0 (from H0)   col 1 (from H1)
        [[  -4.2038       ,     3.1493      ],  <-- row 0 (to O0)
         [   0.1572       ,     0.3311      ],  <-- row 1 (to O1)
         [   1.8735       ,    -3.2946      ]]  <-- row 2 (to O2)
```

This corresponds to the following NN configuration.

```mermaid
%%{init:{'theme': 'neutral'}}%%
graph LR
    %% Style Definitions
    classDef biasStyle fill:none,stroke:none,color:#7f8c8d;

    %% Input Layer (2 nodes)
    subgraph Input Layer [Inputs]
        direction TB
        I0(("I₀"))
        I1(("I₁"))
    end

    %% Hidden Layer (2 nodes)
    subgraph Hidden Layer ["Hidden (ReLU)"]
        direction TB
        H0(("H₀"))
        H1(("H₁"))
    end

    %% Output Layer (3 nodes)
    subgraph Output Layer [Outputs]
        direction TB
        O0(("O₀"))
        O1(("O₁"))
        O2(("O₂"))
    end

    %% Bias Nodes for Hidden Layer
    BH0["Bias: -0.34"]:::biasStyle
    BH1["Bias: 1.24"]:::biasStyle

    %% Bias Nodes for Output Layer
    BO0["Bias: 0.46"]:::biasStyle
    BO1["Bias: 0.96"]:::biasStyle
    BO2["Bias: -0.61"]:::biasStyle

    %% Connections from Input to Hidden Layer Weights
    I0 -->|"3.55"| H0
    I1 -->|"0.25"| H0

    I0 -->|"-1.96"| H1
    I1 -->|"1.48"| H1

    %% Connections from Hidden Layer Biases
    BH0 -.-> H0
    BH1 -.-> H1

    %% Connections from Hidden to Output Layer Weights
    H0 -->|"-4.20"| O0
    H1 -->|"3.15"| O0

    H0 -->|"0.16"| O1
    H1 -->|"0.33"| O1

    H0 -->|"1.87"| O2
    H1 -->|"-3.29"| O2

    %% Connections from Output Layer Biases
    BO0 -.-> O0
    BO1 -.-> O1
    BO2 -.-> O2
```

## Make a prediction

Now that our model is trained, we can use it to make predictions from new data. This is done by passing the model a tensor with normalized petal and sepal widths wrapped up in a tensor, and then passing the model output to `torch.argmax()`.

For example, if the raw petal and sepal width measurements were 0.2 and 3.0, we would first normalize them using the scaler fitted with the training data.

In [12]:
petal_sepal_widths = np.array([.2, 3]).reshape(1, -1)
print("Petal and sepal widths = ", petal_sepal_widths)

normalized_petal_sepal_widths = scaler.transform(petal_sepal_widths)
print("Normalized petal and sepal widths = ", normalized_petal_sepal_widths)

Petal and sepal widths =  [[0.2 3. ]]
Normalized petal and sepal widths =  [[0.04166667 0.41666667]]


c:\Users\Sébastien\Documents\data_science\machine_learning\statsquest_neural_networks\.env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Then we convert `normalized_values` into a tensor and pass it to the model, and pass the output to the `argmax()` function

In [13]:
torch.argmax(
    model(
        torch.tensor(normalized_petal_sepal_widths).type(torch.float32)))

tensor(0)

And the output is 0, meaning that the neural network predicts that the measurements come from **Setosa**.

## Training with the GPU

Using an Intel Iris Xe GPU on my Surface Laptop, we can actually take advantage of it without installing any additional extension, as [explained elsewhere](https://docs.pytorch.org/docs/stable/notes/get_start_xpu.html).

In [14]:
print(torch.xpu.is_available())  # torch.xpu is the API for Intel GPU support

True


In [15]:
# Check if any XPU devices exist first
device_count = torch.xpu.device_count()

if device_count == 0:
    print("No XPU devices found.")
else:
    print(f"Found {device_count} XPU device(s):")
    # Iterate with a standard for loop
    for i in range(device_count):
        print(f"[{i}]: {torch.xpu.get_device_properties(i)}")

Found 1 XPU device(s):
[0]: _XpuDeviceProperties(name='Intel(R) Iris(R) Xe Graphics', platform_name='Intel(R) oneAPI Unified Runtime over Level-Zero', type='gpu', device_id=0x9A49, uuid=8680499a-0100-0000-0002-000000000000, driver_version='1.6.34728', total_memory=14876MB, max_compute_units=96, gpu_eu_count=96, gpu_subslice_count=12, max_work_group_size=512, max_num_sub_groups=64, sub_group_sizes=[8 16 32], has_fp16=1, has_fp64=0, has_atomic64=1)


Now we confirmed GPU is recognized by PyTorch, we can repeat the training using the same model but on another 'device', the DataLoader staying on the CPU. All we would have to do is to configure the 'Trainer' to use the GPU. The `L.Trainer(accelerator="gpu")` is a high-level command that automates all steps when we call `trainer.fit()`.

However, the current version of `lightning` is [not able to use the Intel GPU accelerator](https://github.com/Lightning-AI/pytorch-lightning/issues/20938). But we can take all the individual pieces we defined in the LightningModule and manually arrange them into a "pure PyTorch" training loop. For that, we need to create a new class of model.

First of all, we need to go back one step further, and review the specifications of the DataLoader. Because we haven't provided any btach size, the PyTorch DataLoader used its default, which is `batch_size=1`. Therefore, the model trained using **Stochastic Gradient Descent** (updating weights after every single sample). This is very noisy but can sometimes learn. By setting the argument `batch_size=16`, the model trully trains using **Mini-Batch Gradient Descent** (updating weights 7 times per epoch, or 112/16). We can also try a smaller batch_size (like 8) to create smaller, shorter compute jobs for the GPU, which might prevent it from overheating or timing out.

In [16]:
train_dataloader_gpu = DataLoader(train_dataset, batch_size=16, shuffle=True)

# We also need the test loader for evaluation
test_dataset = TensorDataset(input_test_tensors, label_test_tensors)
test_dataloader = DataLoader(test_dataset, batch_size=16)

Note that the `train_dataloader_gpu` has 112 samples. If our batch_size is 16, this divides perfectly (112 / 16 = 7). The Intel driver "warmed up" by compiling highly optimized code (kernels) for batches of 16. However, the `test_dataloader` has 38 samples, and after the second batch of 16, there is a leftover of 6 samples, and we don't have a compiled kernel for this new, different mismatched shape.

The solution is to add a new `drop_last=True` argument to the definition of the DataLoaders to tell the `test_dataloader` to ignore that last, incomplete batch. This is a standard practice when working with JIT-compilers (like on XPUs or TPUs).

In [17]:
train_dataloader_gpu = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, drop_last=True)

We redefine the model so that it fits the hard PyTorch code. Note that the new class doesn't inherit from `L.LightningModule`, and does not define the optimizer and the training step, like the previous one. The new `nn.Module` class is now just a "blueprint" for the network architecture (the `__init__`) and the data flow (the `forward` pass), all the training logic has been moved out of it.

In [18]:
class MultipleInsOutsGPU(nn.Module):
    def __init__(self):
        super().__init__()
        L.seed_everything(seed=42)
        
        self.net = nn.Sequential(
            nn.Linear(in_features=2, out_features=2),
            nn.ReLU(),
            nn.Linear(in_features=2, out_features=3)
        )
    
    def forward(self, x):
        return self.net(x)

Next step is defining the GPU device.

In [19]:
# Define the device
device = torch.device("xpu")

Finally, we instantiate the model, loss and optimizer as follows.

Note that, the loss would identical when the model's weights are not being updated. This happens when we create the optimizer before we move the model to the GPU. The optimizer locks onto the specific set of parameters (weights and biases) we give it when it's created, trying to update the original model (on the CPU), which will be never used. Therefore, we must create the optimizer **after** we have moved the model to the GPU.

In [20]:
# Define epochs
NUM_EPOCHS = 100

# Instantiate model
model_gpu = MultipleInsOutsGPU()

# Move model to the "xpu" device IN-PLACE
model_gpu.to(device)

# Instantiate loss and optimizer afterwards
criterion = nn.CrossEntropyLoss().to(device)
optimizer = Adam(model_gpu.parameters(), lr=0.001)

print(f"Initiating training on {device} for {NUM_EPOCHS} epochs...")

Seed set to 42


Initiating training on xpu for 100 epochs...


When we call `model.to("xpu")` for the very first time, we are not just moving weights, we are triggering a heavy, one-time initialization process for the entire Intel GPU backend that involves:

- Initializing the oneAPI/Level Zero Runtime: PyTorch has to "wake up" the entire driver stack.
- JIT compilation: this is the main one. The Intel driver (via IPEX or the new PyTorch 2.9 backend) is Just-In-Time compiling the specific GPU code (kernels) needed for the operations in your model (like nn.Linear, nn.ReLU, etc.)

Another issue we might encounter is the GPU is no longer responding for different reasons, such as obverheating, driver crash, or timeout in Windows power management. Also, we may loss information in a Notebook when we re-run the training/testing process.

Therefore it's much cleaner to just train for 100 epochs from the start. It doesn't take much longer to run 100 vs. 10 epochs, the longest waiting time is during the establishment of the connection with the GPU.

In [21]:
# Set model to training mode
model_gpu.train()

MultipleInsOutsGPU(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=3, bias=True)
  )
)

In [22]:
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    for inputs, labels in train_dataloader_gpu:
        
        # Move data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 1. Forward pass
        outputs = model_gpu(inputs)
        
        # 2. Calculate loss
        loss = criterion(outputs, labels)
        
        # 3. Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Print loss once per 10 epochs
    if (epoch + 1) % 10 == 0:
        avg_loss = epoch_loss / len(train_dataloader_gpu)
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}")

print("Training finished.")

Epoch [10/100], Loss: 1.0169
Epoch [20/100], Loss: 0.9831
Epoch [30/100], Loss: 0.9466
Epoch [40/100], Loss: 0.9069
Epoch [50/100], Loss: 0.8652
Epoch [60/100], Loss: 0.8232
Epoch [70/100], Loss: 0.7818
Epoch [80/100], Loss: 0.7419
Epoch [90/100], Loss: 0.7046
Epoch [100/100], Loss: 0.6702
Training finished.


Let's evaluate the GPU model after 100 epochs.

In [23]:
# Set model to evaluation mode
model_gpu.eval()

correct = 0
total = 0

with torch.no_grad():  # We don't need to calculate gradients
    for inputs, labels in test_dataloader:
        # Move test data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Get model outputs
        outputs = model_gpu(inputs)
        
        # Get the prediction (index of the max logit)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)

        # 1. Compare 'predicted' and 'labels' on the GPU
        comparison = (predicted == labels)
        
        # 2. MOVE THE *ENTIRE* BOOLEAN TENSOR TO THE CPU *FIRST*
        comparison_on_cpu = comparison.to("cpu")
        
        # 3. Now, sum the tensor and get the value *on the CPU*.
        #    The CPU knows how to sum a boolean tensor.
        correct += comparison_on_cpu.sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the {total} test samples: {accuracy:.2f} %')

Accuracy on the 32 test samples: 68.75 %


The accuracy after 100 epochs on the GPU is very similar to the accuracy after 10 epochs we obtained from the model with Lightning on the CPU.

The good news is that because we are using a pure PyTorch loop, our GPU model and optimizer objects are still in memory, holding their trained state from the first 100 epochs. To add 900 more epochs, we just need to run the training loop cell again.

In [24]:
NUM_EPOCHS = 1000

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    for inputs, labels in train_dataloader_gpu:
        
        # Move data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 1. Forward pass
        outputs = model_gpu(inputs)
        
        # 2. Calculate loss
        loss = criterion(outputs, labels)
        
        # 3. Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Print loss once every 100 epochs
    if (epoch + 1) % 100 == 0:
        avg_loss = epoch_loss / len(train_dataloader_gpu)
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}")

print("Training finished.")

Epoch [100/1000], Loss: 0.4679
Epoch [200/1000], Loss: 0.3822
Epoch [300/1000], Loss: 0.3217
Epoch [400/1000], Loss: 0.2621
Epoch [500/1000], Loss: 0.1900
Epoch [600/1000], Loss: 0.1536
Epoch [700/1000], Loss: 0.1318
Epoch [800/1000], Loss: 0.1180
Epoch [900/1000], Loss: 0.1085
Epoch [1000/1000], Loss: 0.1018
Training finished.


In [25]:
# Set model to evaluation mode
model_gpu.eval()

correct = 0
total = 0

with torch.no_grad():  # We don't need to calculate gradients
    for inputs, labels in test_dataloader:
        # Move test data to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Get model outputs
        outputs = model_gpu(inputs)
        
        # Get the prediction (index of the max logit)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)

        # 1. Compare 'predicted' and 'labels' on the GPU
        comparison = (predicted == labels)
        
        # 2. MOVE THE *ENTIRE* BOOLEAN TENSOR TO THE CPU *FIRST*
        comparison_on_cpu = comparison.to("cpu")
        
        # 3. Now, sum the tensor and get the value *on the CPU*.
        #    The CPU knows how to sum a boolean tensor.
        correct += comparison_on_cpu.sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the {total} test samples: {accuracy:.2f} %')

Accuracy on the 32 test samples: 93.75 %


After 1000 training epochs, we correctly classified almost 93.75% of the testing data using the NN configured as follows.

In [26]:
# Set the model to evaluation mode first (good practice)
model_gpu.eval()

print("--- Current Model Parameters ---")

# Loop through each parameter (weights and biases)
for name, param in model_gpu.named_parameters():
    
    # We move the data to the CPU for clean printing
    # and use .data to detach it from the graph
    param_data_cpu = param.data.to("cpu")
    
    print(f"Layer: {name}\nValues:\n{param_data_cpu}\n")

--- Current Model Parameters ---
Layer: net.0.weight
Values:
tensor([[ 3.5035,  0.7938],
        [-2.6496,  2.0173]])

Layer: net.0.bias
Values:
tensor([-0.6270,  1.5825])

Layer: net.2.weight
Values:
tensor([[-4.5367,  3.7649],
        [-0.2842,  0.2392],
        [ 3.0659, -3.9450]])

Layer: net.2.bias
Values:
tensor([-0.8396,  2.2043, -1.5938])



Finally we could think about defining functions for training, updating and evaluating the GPU models.

In [27]:
def train_model(model, criterion, optimizer, train_loader, device, num_epochs):
    """
    Trains the model for a given number of epochs.
    Note: This function modifies the model and optimizer in-place.
    """
    print(f"--- Starting training for {num_epochs} epoch(s) on {device} ---")
    
    # Set the model to training mode
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            
            # Move data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # 1. Forward pass
            outputs = model(inputs)
            
            # 2. Calculate loss
            loss = criterion(outputs, labels)
            
            # 3. Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()

        # Print loss at the end of the training epochs
        # (We'll print every 10 epochs, or at the end if fewer than 10)
        if (epoch + 1) % 10 == 0 or (epoch + 1) == num_epochs:
            avg_loss = epoch_loss / len(train_loader)
            print(f"  Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    print("--- Training finished. ---")


def evaluate_model(model, test_loader, device):
    """
    Evaluates the model's accuracy on the test dataset.
    """
    print(f"--- Evaluating model on {device} ---")
    
    # Set model to evaluation mode (e.g., turns off dropout)
    model.eval() 

    correct = 0
    total = 0

    # We don't need to calculate gradients during evaluation
    with torch.no_grad():  
        for inputs, labels in test_loader:
            
            # Move test data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Get model outputs
            outputs = model(inputs)
            
            # Get the prediction (index of the max logit)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            
            # --- Use the CPU workaround for the .sum() bug ---
            comparison = (predicted == labels)
            comparison_on_cpu = comparison.to("cpu")
            correct += comparison_on_cpu.sum().item()

    accuracy = 100 * correct / total
    print(f"  Accuracy on the {total} test samples: {accuracy:.2f} %")
    return accuracy